In [9]:
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel
import rocketcea.cea_obj
import numpy as np
from scipy.optimize import minimize, shgo
import os
import contextlib

In [2]:
allvectors = []               # array for all design vecs, global variable
allobjfun = []                # array for tracking objective function evaluations

In [3]:
def objective(var, cons):
    return (var/cons)**2 / 2
def exterior(var, cons, good_if_less_than=False):
    if good_if_less_than:
        return np.max([0, var/cons - 1])**2 / 2
    else:
        return np.max([0, -(var/cons - 1)])**2 / 2
def exact(var, cons):
    return (var/cons - 1)**2 / 2
def proportion(amount, ratio):
    top = amount * ratio/(1 + ratio)
    bottom = amount * 1/(1 + ratio)
    return top, bottom

In [4]:
def get_propellant_properties(alc_wt, of_ratio, p_ch, exp_ratio, output=False):
    ipa_wt = min(alc_wt, 99)
    ipa_str = '''
    fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(ipa_wt) + '''
    h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    fuel water H 2 O 1  wt%=''' + str(100 - ipa_wt) + '''
    h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    '''
    #eth_ratio = 21.11
    #alc_wt = min(alc_wt, 99.5)
    #eth_wt1, eth_wt2 = proportion(alc_wt, eth_ratio)
    #eth_str = '''
    #fuel C2H5OH(L)   C 2 H 6 O 1       wt%=''' + str(eth_wt1) + '''
    #h,cal=-66370.0      t(k)=298.15       rho,g/cc=0.789
    #fuel C3H8O-2propanol C 3 H 8 O 1    wt%=''' + str(eth_wt2) + '''
    #h,cal=-65133.     t(k)=298.15   rho,g/cc=0.786
    #fuel water H 2 O 1  wt%=''' + str(100 - alc_wt) + '''
    #h,cal=-68308.  t(k)=298.15 rho,g/cc=0.9998
    #'''
    add_new_fuel('LV4_Fuel', ipa_str)
    #add_new_fuel('LV4_Fuel', eth_str)
    if not output:
        PROPELLANT = CEA_Obj(oxName='LOX', fuelName='LV4_Fuel',
                        pressure_units='Pa', temperature_units='K', cstar_units='m/s',
                         density_units='kg/m^3', isp_units='sec', specific_heat_units='J/kg-K')
    else:
        PROPELLANT = rocketcea.cea_obj.CEA_Obj(oxName='LOX', fuelName='LV4_Fuel')
        p_ch *= 1.45e-4
    isp = PROPELLANT.get_Isp(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    IspVac, Cstar, Tc, MW, gamma = PROPELLANT.get_IvacCstrTc_ChmMwGam(Pc=p_ch, MR=of_ratio, eps=exp_ratio)
    string = PROPELLANT.get_full_cea_output(Pc=p_ch, MR=of_ratio, eps=exp_ratio) if output else ''
    return [isp, IspVac, Cstar, Tc, MW, gamma, string]

def prop_cost(x):
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    #ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma, string = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    merit = -objective(IspVac, 250) + 100* exact(p_ch, 2413166) + 100 * exact(exp_ratio, 4.5495)
    return merit

def prop_opt(x0):
    #res = minimize(prop_cost, x0, method='nelder-mead', options={'disp': True, 'adaptive':True})
    res = shgo(prop_cost,# options={'disp':True},
               n=50, iters=2, sampling_method='sobol',
               bounds=[*zip([0.1,0.1,0.1, 3],
                                [100,10,2413166*2,7])],
              minimizer_kwargs={'method':'Nelder-Mead', 'options':{'adaptive':True}})
    x = res.x
    return x

def prop_run():
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        x = prop_opt([64.8, 1.3, 2413166, 5.988])
        ipa_wt, of_ratio, p_ch, exp_ratio = x
        ipa_wt = min(ipa_wt, 99)
        #ipa_wt = 64.8
        isp, IspVac, Cstar, Tc, MW, gamma, string = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio, True)
        isp, IspVac, Cstar, Tc, MW, gamma, _ = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    if __name__ == '__main__' and not '__file__' in globals():
        print('Alcohol Wt %: ', ipa_wt)
        print('OF ratio: ', of_ratio)
        print('P_ch (Pa): ', p_ch)
        print('Expansion ratio: ', exp_ratio)
        print()
        print('Vacuum ISP (s): ', IspVac)
        print('Chamber Temp (K): ', Tc)
        print('Molar Wt (1/n): ', MW)
        print('Spec Heat: ', gamma)
    global PROPELLANT_SET
    PROPELLANT_SET = True
    return ipa_wt, of_ratio, p_ch, Tc, MW, gamma, string
    

In [5]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 100 times
reading cea isp data files for LOX / LV4_Fuel 200 times
reading cea isp data files for LOX / LV4_Fuel 300 times
reading cea isp data files for LOX / LV4_Fuel 400 times
reading cea isp data files for LOX / LV4_Fuel 500 times
reading cea isp data files for LOX / LV4_Fuel 600 times
reading cea isp data files for LOX / LV4_Fuel 700 times
reading cea isp data files for LOX / LV4_Fuel 800 times
reading cea isp data files for LOX / LV4_Fuel 900 times
Alcohol Wt %:  99
OF ratio:  1.769820563700904
P_ch (Pa):  2413422.4914040067
Expansion ratio:  4.556627500008137

Vacuum ISP (s):  295.0410396969782
Chamber Temp (K):  3343.3875797367878
Molar Wt (1/n):  22.51063136308655
Spec Heat:  1.1306817834913114


In [6]:
if __name__ == '__main__' and not '__file__' in globals(): prop_run()

reading cea isp data files for LOX / LV4_Fuel 1000 times
reading cea isp data files for LOX / LV4_Fuel 1100 times
reading cea isp data files for LOX / LV4_Fuel 1200 times
reading cea isp data files for LOX / LV4_Fuel 1300 times
reading cea isp data files for LOX / LV4_Fuel 1400 times
reading cea isp data files for LOX / LV4_Fuel 1500 times
reading cea isp data files for LOX / LV4_Fuel 1600 times
reading cea isp data files for LOX / LV4_Fuel 1700 times
reading cea isp data files for LOX / LV4_Fuel 1800 times
reading cea isp data files for LOX / LV4_Fuel 1900 times
Alcohol Wt %:  99
OF ratio:  1.769820563700904
P_ch (Pa):  2413422.4914040067
Expansion ratio:  4.556627500008137

Vacuum ISP (s):  295.0410396969782
Chamber Temp (K):  3343.3875797367878
Molar Wt (1/n):  22.51063136308655
Spec Heat:  1.1306817834913114


In [7]:
def propellant_cost(x, chamber_pressure):
    ipa_wt, of_ratio, p_ch, exp_ratio = x
    #ipa_wt = 64.8
    isp, IspVac, Cstar, Tc, MW, gamma, string = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    merit = -objective(IspVac, 250) + 100 * exact(p_ch, chamber_pressure) + 100 * exact(exp_ratio, 4.5495)
    return merit

def propellant_optimizer(chamber_pressure):
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        res = minimize(propellant_cost, [64.8, 1.3, chamber_pressure, 4.5495], args=(chamber_pressure), method='nelder-mead', options={'adaptive':True})
        ipa_wt, of_ratio, p_ch, exp_ratio = res.x
        #ipa_wt = 64.8
        ipa_wt = min(ipa_wt, 99)
        isp, IspVac, Cstar, Tc, MW, gamma, _ = get_propellant_properties(ipa_wt, of_ratio, p_ch, exp_ratio)
    return ipa_wt, of_ratio, p_ch, Tc, MW, gamma, _

In [10]:
#%%timeit -n 1 -r 1 pass
if __name__ == '__main__' and not '__file__' in globals():
    ipa_wt, of_ratio, p_ch, Tc, MW, gamma, string = propellant_optimizer(2413166)